In [ ]:
from brian2 import *
from model_explorer_jupyter import *
import ipywidgets as ipw

def normed(X):
    return X/amax(abs(X))

In [ ]:
def adapted_signal_inh(fm_array, taue, taui, taua, alpha, beta, gamma, level):
    k = (1-alpha)/taua
    R = alpha/taua
    gain = 10**(level/20.)
    eqs = '''
    A = (gain*0.5*(1-cos(2*pi*fm*t)))**gamma : 1
    dQ/dt = -k*Q*A+R*(1-Q) : 1
    I = A*Q : 1
    dAe/dt = (I-Ae)/taue : 1
    dAi/dt = (I-Ai)/taui : 1
    out = Ae-beta*Ai : 1
    fm : Hz
    '''
    G = NeuronGroup(len(fm_array), eqs, method='euler', dt=0.1*ms)
    G.fm = fm_array
    M = StateMonitor(G, True, record=True)
    Network(G, M).run(1*second)
    return M.t[:], M.out[:]

def peak_ipd_inh(fm_array, taue, taui, taua, alpha, beta, gamma, level):
    T, out = adapted_signal_inh(fm_array, taue, taui, taua, alpha, beta, gamma, level)
    peak_phases = []
    peak_fr = []
    for out, fm in zip(out, fm_array):
        n = int(round(0.5*second*fm))
        I = T>n/fm
        TI = T[I]
        out = out[I]
        peak = TI[argmax(out)]
        peak_phase = (peak*2*pi*fm) % (2*pi)
        peak_phases.append(peak_phase)
        peak_fr.append(amax(out))
    return array(peak_phases), array(peak_fr)

def show_peak_ipd_inh(taue=0.1, taui=0.5, taua=5, alpha=0.2, beta=1.0, gamma=1, level=0.0):
    fm_array = array([4, 8, 16, 32, 64])*Hz
    phi, fr = peak_ipd_inh(fm_array, taue*ms, taui*ms, taua*ms, alpha, beta, gamma, level)
    figure(figsize=(12, 5))
    subplot(121)
    plot(fm_array, phi*180/pi, '-o')
    plot(fm_array, [37, 40, 62, 83, 115], '--r')
    grid()
    ylim(0, 180)
    xlabel('Modulation frequency (Hz)')
    ylabel('Extracted phase (deg)')
    subplot(122)
    plot(fm_array, fr/amax(fr), '-o')
    ylim(0, 1)
    xlabel('Modulation frequency (Hz)')
    ylabel('Relative peak firing rate')
    tight_layout()
    
w = ipw.interactive(show_peak_ipd_inh,
        taue=ipw.FloatSlider(min=0.1, max=10, step=0.1, value=0.1,
            description=r"Excitatory filtering time constant $\tau_e$ (ms)"),
        taui=ipw.FloatSlider(min=0.1, max=10, step=0.1, value=0.5,
            description=r"Inhibitory filtering time constant $\tau_i$ (ms)"),
        taua=ipw.FloatSlider(min=0.1, max=10, step=0.1, value=5,
            description=r"Adaptation time constant $\tau_a$ (ms)"),
        alpha=ipw.FloatSlider(min=0.1, max=1, step=0.01, value=0.2,
            description=r"Adaptation strength $\alpha$"),
        beta=ipw.FloatSlider(min=0.1, max=2, step=0.01, value=1.0,
            description=r"Inhibition strength $\beta$"),
        gamma=ipw.FloatSlider(min=0.1, max=1, step=0.01, value=1.0,
            description=r"Compression power $\gamma$"),
        level=ipw.FloatSlider(min=-90, max=90, step=5, value=0,
            description=r"Relative sound level $L$ (dB)"),
        )

for child in w.children:
    if isinstance(child, ipw.ValueWidget):
        child.layout.width = '100%'
        child.style = {'description_width': '30%'}
        child.continuous_update = False

display(w)